# Multiple Linear Regression

The multiple linear regression is similar to the simple linear regression, in that it is described by a straight line; however, the difference is that it depends on more than a single feature:

$
y=b_{0}+b_{1}x_{1}+\ldots+b_{n}x_{n},
$

where $n=1,2,\ldots,n$.

We will build a machine learning model that will find the best fit line given multiple features under the assumption that, within the data, we have:
+ linearity
+ homoscedasticity (all random variables in the vector have the same finite variance)
+ multivariate normality
+ independent of errors
+ sans multicollinearity

We will not go over ensuring the data has these properties as this is a toy box exercise.

## The Business Understanding and The Data

For this exercise we will explore a business problem with the 50 Startups dataset. We are given profit/loss statement for 50 comapnies and we are assuming that a VC firm has tasked us with looking into these companies and create a model that will help them determine which types of companies they should invest in. In our data set, Profit is the response variable that depends on R&D Spend, Administration, State, Marketing Spend, and State as features.

The firm is not necessarily looking for which companies in this set to invest in; rather, they are looking for features they should consider based on this sample. Do companies in Florida perform better than those in California? Or does a company that invests more heavily in R&D turn more profit than one that invested heavily in marketing? They want to know how to assess companies. Where, and which, types of companies should they spend on?

So, can we determine a linear dependency between all the features and the response, Profit?

Let's start answering this by bringing in and inspecting the data.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load the data as a dataframe
df = pd.read_csv('data/50_Startups.csv')

# inspect shape and first rows of data
print(df.shape, '\n', df.head())

(50, 5) 
    R&D Spend  Administration  Marketing Spend       State     Profit
0  165349.20       136897.80        471784.10    New York  192261.83
1  162597.70       151377.59        443898.53  California  191792.06
2  153441.51       101145.55        407934.54     Florida  191050.39
3  144372.41       118671.85        383199.62    New York  182901.99
4  142107.34        91391.77        366168.42     Florida  166187.94


We know that Profit is our response variable, so let's look a little more at our features. W have:
+ $x_{1}$ = R&D Spend
+ $x_{2}$ = Administration
+ $x_{3}$ = Marketing Spend
+ $x_{4}$ = State

We can see that the the first three features are pretty straight forward, but the State column is obviously categorical. This means we need to introduce dummy variables. For each category in the column we will create a new column. This implies our regression model will look more like:

$
y=b_{0}+b_{1}x_{1}+b_{2}x_{2}+b_{3}x_{3}+b_{4}D_{1}+\ldots+b_{n+2}D_{n-1},
$

where $D_{1}$ is the first dummy variable and $n$ is the total number of categories required by that feature. Remember that, in order to avoid the dummy variable trap, we will want to ensure we have one less dummy variable than the total number of categories in our categorical feature (hence $n-1$). Note that the $n+2$ takes into account, not only the one less dummy variable, but also $b_{1}$, $b_{2}$, and $b_{3}$.



## How To Build A Model With More Than One Feature

Since the this model has more than one feature, we must consider the features before cramming them into an algorithm and demand it build us an army worthy of Mordor. We have to actually *build* the model by analyzing the features and selecting which ones are best to include in the model. This is because all the features are potential predictors of the response variable and, according to the garbage in, garbage out principle, if some of the predictors are garbage and we include them, our model will give us garbage. Another reason is that, when it comes to explaining our model to someone who doesn't have the knowledge we do in our field (like a client), it is easier to explain if we don't have a thousand predictors to talk about.

There are a number of different ways to do this. We could use:

+ Use them all
+ Backward elimination
+ Forward selection
+ Bidirectional elimination
+ Score comparison

Each has it's own ups and downs. Let's get a little information on these.

### Use 'Em All

Basically, use every single one of the features in the model. Not recommended, but if you have knowledge about the features and know that they all belong in the model, go for it. Otherwise, do this if you just have to, like if you are told to by "the boss," or is just a part of the biz.

There is another reason to do this: when you're preparing for the backward elimination strategy.

### Backward Elimination

This is where we take a significance level (like, say, 0.05) that will be the threshold for keeping features in the model. Then, taking all the features, fit the model. Once the model has been fitted, consider predictor with the highest *p-value* and if it is higher than you're significance level, drop it. Next, fit the model again, without that feature. Rinse and repeat with the next feature that has the highest *p-value*. When there ain't no more feature with *p-value* greater than your significance level, there you go.

### Forward Selection

In forward selection, we select a significance level to *enter* the model, then fit a simple regression model with *every single feature* independently. When we have done this, we keep only the one feature that has the lowest *p-value*. Now, we fit all possible models with that feature *and* one additional feature, for each other feature. Consider the one variable that has the lowest *p-value* that was not the the kept before with the intention of continuing this process. building the model one variable at a time. We call it quits when we no longer have a feature whose *p-value* is less than our significance level.

### Bidirectional Elimination

This is, in essence, a combination of backward elimination and forward selection. We start by setting a significance level for entering the model and another for dropping out of the model. The we kick off with fitting a simple regression model with each feature, noting that new features must have a *p-value* greater than our entry significance level. Next we perform all the steps involved in backward elimination, noting that features will get dropped if their *p-value* is less then the siginificance level required to stay in the model. Keep doing this until no new features can be added *and* no old features can be kicked out.

### Score Comparison

This one is interesting. It the most resource consuming in that we consider all possible regression models and select the best one based on a set of criterion I call the "the special goodness." Understand that the the claim that this is resource intensive is because trying out all possible regression models is a combination problem. As in all possible models is $2^{N}-1$ models. To put that in perspective, if you have a data set with 14 feature variables, you then have 16,383 possible regression models to try.

Now that we have some plans on how to treat the data, let's get to it!

In [3]:
# separate the data into response and feature dataframes
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [4]:
# inspect these matrices
print('X\n', X, '\n', 'y\n', y)

X
 [[165349.2 136897.8 471784.1 'New York']
 [162597.7 151377.59 443898.53 'California']
 [153441.51 101145.55 407934.54 'Florida']
 [144372.41 118671.85 383199.62 'New York']
 [142107.34 91391.77 366168.42 'Florida']
 [131876.9 99814.71 362861.36 'New York']
 [134615.46 147198.87 127716.82 'California']
 [130298.13 145530.06 323876.68 'Florida']
 [120542.52 148718.95 311613.29 'New York']
 [123334.88 108679.17 304981.62 'California']
 [101913.08 110594.11 229160.95 'Florida']
 [100671.96 91790.61 249744.55 'California']
 [93863.75 127320.38 249839.44 'Florida']
 [91992.39 135495.07 252664.93 'California']
 [119943.24 156547.42 256512.92 'Florida']
 [114523.61 122616.84 261776.23 'New York']
 [78013.11 121597.55 264346.06 'California']
 [94657.16 145077.58 282574.31 'New York']
 [91749.16 114175.79 294919.57 'Florida']
 [86419.7 153514.11 0.0 'New York']
 [76253.86 113867.3 298664.47 'California']
 [78389.47 153773.43 299737.29 'New York']
 [73994.56 122782.75 303319.26 'Florida']
 [67

Let's encode that State column with our One Hot Encoding!

In [5]:
# encoding the categorical feature
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# the encoding
labelencoder_X = LabelEncoder()
X[:,-1] = labelencoder_X.fit_transform(X[:,-1]) 

onehotencoder = OneHotEncoder(categorical_features=[3])
X = onehotencoder.fit_transform(X).toarray()

In [6]:
# inspect encoding of categorical feature
print(pd.DataFrame(X).head())

     0    1    2          3          4          5
0  0.0  0.0  1.0  165349.20  136897.80  471784.10
1  1.0  0.0  0.0  162597.70  151377.59  443898.53
2  0.0  1.0  0.0  153441.51  101145.55  407934.54
3  0.0  0.0  1.0  144372.41  118671.85  383199.62
4  0.0  1.0  0.0  142107.34   91391.77  366168.42


Excellent. But now that we have this encoded properly, let's make sure we avoind the dummy variable trap.

In [7]:
# avoid dummy variable trap
X = X[:,1:]  # simply remove the first column of the features data set

In [8]:
# split into traiinng and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
# inspect the training set
print('X\n', X_train, '\n', 'y\n', y_train)

X
 [[1.0000000e+00 0.0000000e+00 5.5493950e+04 1.0305749e+05 2.1463481e+05]
 [0.0000000e+00 1.0000000e+00 4.6014020e+04 8.5047440e+04 2.0551764e+05]
 [1.0000000e+00 0.0000000e+00 7.5328870e+04 1.4413598e+05 1.3405007e+05]
 [0.0000000e+00 0.0000000e+00 4.6426070e+04 1.5769392e+05 2.1079767e+05]
 [1.0000000e+00 0.0000000e+00 9.1749160e+04 1.1417579e+05 2.9491957e+05]
 [1.0000000e+00 0.0000000e+00 1.3029813e+05 1.4553006e+05 3.2387668e+05]
 [1.0000000e+00 0.0000000e+00 1.1994324e+05 1.5654742e+05 2.5651292e+05]
 [0.0000000e+00 1.0000000e+00 1.0002300e+03 1.2415304e+05 1.9039300e+03]
 [0.0000000e+00 1.0000000e+00 5.4205000e+02 5.1743150e+04 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00 6.5605480e+04 1.5303206e+05 1.0713838e+05]
 [0.0000000e+00 1.0000000e+00 1.1452361e+05 1.2261684e+05 2.6177623e+05]
 [1.0000000e+00 0.0000000e+00 6.1994480e+04 1.1564128e+05 9.1131240e+04]
 [0.0000000e+00 0.0000000e+00 6.3408860e+04 1.2921961e+05 4.6085250e+04]
 [0.0000000e+00 0.0000000e+00 7.8013110e+04 1.21

In [10]:
# inspect the test set
print('X\n', X_test, '\n', 'y\n', y_test)

X
 [[1.0000000e+00 0.0000000e+00 6.6051520e+04 1.8264556e+05 1.1814820e+05]
 [0.0000000e+00 0.0000000e+00 1.0067196e+05 9.1790610e+04 2.4974455e+05]
 [1.0000000e+00 0.0000000e+00 1.0191308e+05 1.1059411e+05 2.2916095e+05]
 [1.0000000e+00 0.0000000e+00 2.7892920e+04 8.4710770e+04 1.6447071e+05]
 [1.0000000e+00 0.0000000e+00 1.5344151e+05 1.0114555e+05 4.0793454e+05]
 [0.0000000e+00 1.0000000e+00 7.2107600e+04 1.2786455e+05 3.5318381e+05]
 [0.0000000e+00 1.0000000e+00 2.0229590e+04 6.5947930e+04 1.8526510e+05]
 [0.0000000e+00 1.0000000e+00 6.1136380e+04 1.5270192e+05 8.8218230e+04]
 [1.0000000e+00 0.0000000e+00 7.3994560e+04 1.2278275e+05 3.0331926e+05]
 [1.0000000e+00 0.0000000e+00 1.4210734e+05 9.1391770e+04 3.6616842e+05]] 
 y
 [103282.38 144259.4  146121.95  77798.83 191050.39 105008.31  81229.06
  97483.56 110352.25 166187.94]


Now that we have created the training and test sets, it's time to...

## Fit The Model

Recall from above that there are some startegies for building the model with choosing the best features. We will run through this with choosing:

### All The Features

In [11]:
# fit the multiple linear regression model to the training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

The model has been fit to the training set which leaves us with testing the model's peformance. Let's do it!

## Testing Model Performance

We will create the vector of predictions. This is the vector populated with predictions of our test set, made by the model we just created.

In [12]:
# prediciting test observations
y_pred = regressor.predict(X_test)

In [13]:
# inspect the predicted observation and the actual observations
y_test_show = np.array(y_test)
y_pred_show = np.array(y_pred)
y_show = np.column_stack((y_test_show,y_pred_show))

print('********Test & Predicted********\n',y_show)

********Test & Predicted********
 [[103282.38       103015.20159796]
 [144259.4        132582.27760815]
 [146121.95       132447.73845175]
 [ 77798.83        71976.09851258]
 [191050.39       178537.48221056]
 [105008.31       116161.24230166]
 [ 81229.06        67851.69209676]
 [ 97483.56        98791.73374687]
 [110352.25       113969.43533013]
 [166187.94       167921.06569551]]


Not too shabby!

So that was when we choose all the features to build the model. What happens if we decided to use:

### Backward Elimination

Since not everyone is a fan of reinventing the wheel when we just need to get the damn thing done, we will call upon the Stasmodel package to help us out with this. Otherwise, could implement it ourselves, which is always an option, especially if you wish to fine tune it, or you have a different implementation strategy. You do you!

One thing about this package though, it does not include the $b_{0}$ constant that our model requires. We will need to take care of that.

In [14]:
# import the statsmodels package
import statsmodels.formula.api as sm

In [15]:
# add column of ones that corresponds to the b_0 constant
X = np.append(arr=np.ones((50,1)).astype(int), values=X, axis=1)

In [16]:
# inspect the X array to ensure we did that column add correctly
print(pd.DataFrame(X).head())

     0    1    2          3          4          5
0  1.0  0.0  1.0  165349.20  136897.80  471784.10
1  1.0  0.0  0.0  162597.70  151377.59  443898.53
2  1.0  1.0  0.0  153441.51  101145.55  407934.54
3  1.0  0.0  1.0  144372.41  118671.85  383199.62
4  1.0  1.0  0.0  142107.34   91391.77  366168.42


In [17]:
# new matrix of feature choosing them with backward elimination
# columns explicitely called so we can "see" what we're removing as we build
X_optimal = X[:,[0,1,2,3,4,5]]

In [18]:
# create new regressor, ordinary least squares: regressor_OLS
regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()

In [19]:
# use summary function from statsmodels to compare the p-values of the features in the trained model
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Fri, 13 Jul 2018   Prob (F-statistic):           1.34e-27
Time:                        22:52:12   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.0

 So we are seeing the *p-values* of the features in the model. As we can see, the $x_{2}$ variable is *way* above the 0.05 siginificance level we cited in the explanation above. We need to let it go.

In [20]:
# matrix of feature choosing them with backward elimination
# columns explicitely called so we can "see" what we're removing as we build
X_optimal = X[:,[0,1,3,4,5]]

# create new regressor, ordinary least squares: regressor_OLS
regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()

# use summary function from statsmodels to compare the p-values of the features in the trained model
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Fri, 13 Jul 2018   Prob (F-statistic):           8.49e-29
Time:                        22:52:12   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.0

Cut $x_{1}$!

In [21]:
# matrix of feature choosing them with backward elimination
# columns explicitely called so we can "see" what we're removing as we build
X_optimal = X[:,[0,3,4,5]]

# create new regressor, ordinary least squares: regressor_OLS
regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()

# use summary function from statsmodels to compare the p-values of the features in the trained model
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Fri, 13 Jul 2018   Prob (F-statistic):           4.53e-30
Time:                        22:52:12   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.0

And out with the $x_{2}$.

In [22]:
# matrix of feature choosing them with backward elimination
# columns explicitely called so we can "see" what we're removing as we build
X_optimal = X[:,[0,3,5]]

# create new regressor, ordinary least squares: regressor_OLS
regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()

# use summary function from statsmodels to compare the p-values of the features in the trained model
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Fri, 13 Jul 2018   Prob (F-statistic):           2.16e-31
Time:                        22:52:12   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.0

Ok, let's slow down here. We have used backward elimination to eliminate all features except for two. As you can see in the table above, the last feature has a $p-value$ that is slightly above the significance level of 0.05, which means we drop it like it's hot, according to the process. This may not end up being the optimal model we are about to make if we cut it. We will explore another feature selection startegy that may end up keeping more than just a single feature, but for the sake of this demonstration, we will follow the algorithm and cut it.

In [23]:
# matrix of feature choosing them with backward elimination
# columns explicitely called so we can "see" what we're removing as we build
X_optimal = X[:,[0,3]]

# create new regressor, ordinary least squares: regressor_OLS
regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()

# use summary function from statsmodels to compare the p-values of the features in the trained model
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Fri, 13 Jul 2018   Prob (F-statistic):           3.50e-32
Time:                        22:52:12   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.0

According to the backward elimination strategy, the feature, $x_{1}$, or R&D Spending, is a huge predictor of Profit! If we built the model with that as the feature, we might end up with an optimal model. But I suggest you try another strategy first to be sure. Have fun!

As a bonus, here is script, from Hadelin de Ponteves, that automatically handles backward elimination using a significance level of 0.05, *and* the adjusted R-squared value:

In [24]:
#import statsmodels.formula.api as sm
#def backwardElimination(x, SL):
#    numVars = len(x[0])
#    temp = np.zeros((50,6)).astype(int)
#    for i in range(0, numVars):
#        regressor_OLS = sm.OLS(y, x).fit()
#        maxVar = max(regressor_OLS.pvalues).astype(float)
#        adjR_before = regressor_OLS.rsquared_adj.astype(float)
#        if maxVar > SL:
#            for j in range(0, numVars - i):
#                    temp[:,j] = x[:, j]
#                    x = np.delete(x, j, 1)
#                    tmp_regressor = sm.OLS(y, x).fit()
#                    adjR_after = tmp_regressor.rsquared_adj.astype(float)
#                    if (adjR_before >= adjR_after):
#                        x_rollback = np.hstack((x, temp[:,[0,j]]))
#                        x_rollback = np.delete(x_rollback, j, 1)
#                        print (regressor_OLS.summary())
#                        return x_rollback
#                    else:
#                        continue
#    regressor_OLS.summary()
#    return x
# 
#SL = 0.05
#X_optimal = X[:, [0, 1, 2, 3, 4, 5]]
#X_Modeled = backwardElimination(X_opt, SL)